In [1]:
from itertools import product

import pandas as pd
import numpy as np

# Funciones

In [2]:
def generate_options(type_pc, cpu_range, cpu_gen_range, cpu_series, ram_range, hdd, ssd, gpu_serie=None, gpu_model=None, enable_ti=False, without_gpu=False):
    cpus = [
        f"{cpu}-{gen}-{serie}"
        for cpu in range(3,8,2)
        for gen in range(5,12)
        for serie in cpu_series
    ]

    if not without_gpu:

        gpu_without_ti = [
            f"{serie}{model}"
            for serie in gpu_serie
            for model in gpu_model    
        ]
        gpu_with_ti = []
        if enable_ti:
            gpu_with_ti = [f"{gpu}-TI" for gpu in gpu_without_ti]
        gpu = gpu_without_ti + gpu_with_ti
    else:
        gpu = [-1]

    ram = list(ram_range)

    return list(product(cpus, ram, gpu, hdd, ssd, [type_pc]))

# Creación 

In [3]:
pcs = []

## Ofimatica

In [4]:
pcs = [
    *pcs,
    *generate_options("Ofimatica", range(3,6,2), range(5,8), ["Q", "B", "U"], range(2, 9), [500, 750, 1024], [0, 128, 256], without_gpu=True)
]

## Gamer

In [5]:
pcs = [
    *pcs,
    *generate_options(
        "Gamer",
        range(5,10,2),
        range(8,13),
        ["H", "Z"],
        range(8, 33, 4),
        [1024*i for i in range(1,5)],
        [2**i for i in range(7, 11)],
        [10, 20, 30],
        [30, 50, 60, 70, 80],
        True
    )
]

## Diseño

In [6]:
pcs = [
    *pcs,
    *generate_options(
        "Diseño",
        range(5,10,2),
        range(7,13),
        ["H", "Z"],
        range(16, 128, 4),
        [1024*i for i in range(2,11)],
        [2**i for i in range(10, 13)],
        [10, 20, 30],
        [30, 50, 60, 70, 80],
        True
    )
]

# Prepocesamiento

In [7]:
df_base = pd.DataFrame(pcs, columns=["CPU", "RAM", "GPU", "HDD", "SSD", "TARGET"])

In [8]:
df = df_base.copy(deep=True)

In [9]:
df["CPU_GEN"] = df["CPU"].apply(lambda cpu: cpu.split("-")[1])
df["CPU_SERIE"] = df["CPU"].apply(lambda cpu: cpu.split("-")[-1])
df["CPU"] = df["CPU"].apply(lambda cpu: cpu.split("-")[0])

In [10]:
df["GPU_TI"] = df["GPU"].apply(lambda gpu: 1 if "TI" in str(gpu) else 0)
df["GPU"] = df["GPU"].apply(lambda gpu: int(gpu) if str(gpu).isnumeric() or gpu == -1 else int(gpu.split("-")[0]))

In [11]:
COLUMNS_ORDER = ["CPU", "CPU_GEN", "CPU_SERIE", "RAM", "GPU", "GPU_TI", "HDD", "SSD", "TARGET"]

In [12]:
df = df[COLUMNS_ORDER]

In [13]:
df

,CPU,CPU_GEN,CPU_SERIE,RAM,GPU,GPU_TI,HDD,SSD,TARGET
0,3,5,Q,2,-1,0,500,0,Ofimatica
1,3,5,Q,2,-1,0,500,128,Ofimatica
2,3,5,Q,2,-1,0,500,256,Ofimatica
3,3,5,Q,2,-1,0,750,0,Ofimatica
4,3,5,Q,2,-1,0,750,128,Ofimatica
...,...,...,...,...,...,...,...,...,...
1097644,7,11,Z,124,3080,1,9216,2048,Diseño
1097645,7,11,Z,124,3080,1,9216,4096,Diseño
1097646,7,11,Z,124,3080,1,10240,1024,Diseño
1097647,7,11,Z,124,3080,1,10240,2048,Diseño


In [14]:
df.to_csv("/work/games_seeker/data/interim/types_of_pc.csv", sep="|", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=07e29549-6cbf-4e88-b69c-0fc48f2d6023' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>